<a href="https://colab.research.google.com/github/Alena-Grebeniuk/ML-and-DA/blob/main/Lab06_LinearClassification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Linear Classification

In this lab you will implement parts of a linear classification model using the regularized empirical risk minimization principle. By completing this lab and analysing the code, you gain deeper understanding of these type of models, and of gradient descent.


## Problem Setting

The dataset describes diagnosing of cardiac Single Proton Emission Computed Tomography (SPECT) images. Each of the patients is classified into two categories: normal (1) and abnormal (0). The training data contains 80 SPECT images from which 22 binary features have been extracted. The goal is to predict the label for an unseen test set of 187 tomography images.

In [1]:
import urllib.request
import pandas as pd
import numpy as np
# for auto-reloading external modules
# see http://stackoverflow.com/questions/1907993/autoreload-of-modules-in-ipython
%load_ext autoreload
%autoreload 2

testfile = urllib.request.URLopener()
testfile.retrieve("http://archive.ics.uci.edu/ml/machine-learning-databases/spect/SPECT.train", "SPECT.train")
testfile.retrieve("http://archive.ics.uci.edu/ml/machine-learning-databases/spect/SPECT.test", "SPECT.test")

df_train = pd.read_csv('SPECT.train',header=None)
df_test = pd.read_csv('SPECT.test',header=None)

train = df_train.values
test = df_test.values

y_train = train[:,0]
X_train = train[:,1:]
y_test = test[:,0]
X_test = test[:,1:]

### Exercise 1

Analyze the function learn_reg_ERM(X,y,lambda) which for a given $n\times m$ data matrix $\textbf{X}$ and binary class label $\textbf{y}$ learns and returns a linear model $\textbf{w}$.
The binary class label has to be transformed so that its range is $\left \{-1,1 \right \}$.
The trade-off parameter between the empirical loss and the regularizer is given by $\lambda > 0$.
To adapt the learning rate the Barzilai-Borwein method is used.

Try to understand each step of the learning algorithm and comment each line.


In [2]:
def learn_reg_ERM(X,y,lbda):
    max_iter = 200
    e  = 0.001
    alpha = 1.

    w = np.random.randn(X.shape[1]);
    for k in np.arange(max_iter):
        h = np.dot(X,w)
        l,lg = loss(h, y)
        print ('loss: {}'.format(np.mean(l)))
        r,rg = reg(w, lbda)
        g = np.dot(X.T,lg) + rg
        if (k > 0):
            alpha = alpha * (np.dot(g_old.T,g_old))/(np.dot((g_old - g).T,g_old))
        w = w - alpha * g
        if (np.linalg.norm(alpha * g) < e):
            break
        g_old = g
    return w

This function seems to implement a regularized empirical risk minimization (ERM) algorithm with a linear model. Let's break down each step:

1. `max_iter = 200`: Sets the maximum number of iterations for the optimization process. If the algorithm doesn't converge within these iterations, it stops.

2. `e = 0.001`: Defines a threshold for the norm of the gradient. If the norm of the gradient falls below this threshold, the optimization process is considered converged, and the algorithm stops.

3. `alpha = 1.`: Initializes the learning rate. It's a common practice to start with a default value.

4. `w = np.random.randn(X.shape[1])`: Initializes the weight vector randomly. The dimensionality of the weight vector matches the number of features in the dataset.

5. `for k in np.arange(max_iter):`: Starts a loop over the maximum number of iterations.

6. `h = np.dot(X, w)`: Computes the linear combination of the input features and the weight vector, representing the predicted values.

7. `l, lg = loss(h, y)`: Computes the loss and its gradient with respect to the predicted values `h` and the true labels `y`. This function seems to be external to the provided code.

8. `r, rg = reg(w, lbda)`: Computes the regularization term and its gradient with respect to the weight vector `w` and the regularization parameter `lbda`. This function also seems to be external.

9. `g = np.dot(X.T, lg) + rg`: Computes the total gradient of the loss function by summing the gradient of the loss term and the gradient of the regularization term.

10. `if (k > 0): ...`: This block adapts the learning rate `alpha` using the Barzilai-Borwein method. This method adjusts the learning rate based on the gradient descent steps taken so far.

11. `w = w - alpha * g`: Updates the weight vector using gradient descent.

12. `if (np.linalg.norm(alpha * g) < e): break`: Checks if the norm of the gradient times the learning rate falls below the threshold `e`. If it does, the algorithm terminates as it indicates convergence.

13. `g_old = g`: Stores the gradient for the next iteration.

14. `return w`: Returns the learned weight vector, which represents the linear model.

Overall, this function implements a regularized empirical risk minimization algorithm with gradient descent, where the learning rate is adapted using the Barzilai-Borwein method, and termination occurs when the gradient falls below a certain threshold.

### Exercise 2

Fill in the code for the function loss(h,y) which computes the hinge loss and its gradient.
This function takes a given vector $\textbf{y}$ with the true labels $\in \left \{-1,1\right \}$ and a vector $\textbf{h}$ with the function values of the linear model as inputs. The function returns a vector $\textbf{l}$ with the hinge loss $\max(0, 1 − y_{i} h_{i})$ and a vector $\textbf{g}$ with the gradients of the hinge loss w.r.t $\textbf{h}$. (Note: The partial derivative of the hinge loss with respect to $\textbf{h}$  is $g_{i} = −y $ if $l_{i} > 0$, else $g_{i} = 0$)

In [3]:
import numpy as np

def loss(h, y):
    l = np.maximum(0, 1 - y * h)  # hinge loss
    g = -y * (l > 0)             # gradient of hinge loss
    return l, g


Explanation:

l = np.maximum(0, 1 - y * h): This line computes the hinge loss element-wise. It takes the maximum of 0 and 1 - y * h. If y * h >= 1, the hinge loss is 0; otherwise, it's 1 - y * h.

g = -y * (l > 0): This line computes the gradient of the hinge loss. If l > 0, meaning the hinge loss is non-zero, the gradient is -y. Otherwise, it's 0.

This function computes both the hinge loss l and its gradient g and returns them.

### Exercise 3

Fill in the code for the function reg(w,lambda) which computes the $\mathcal{L}_2$-regularizer and the gradient of the regularizer function at point $\textbf{w}$.


$$r = \frac{\lambda}{2} \textbf{w}^{T}\textbf{w}$$

$$g = \lambda \textbf{w}$$

In [4]:
import numpy as np

def reg(w, lbda):
    r = 0.5 * lbda * np.dot(w.T, w)  # ℓ2-regularizer
    g = lbda * w                      # Gradient of ℓ2-regularizer
    return r, g


Explanation:

r = 0.5 * lbda * np.dot(w.T, w): This line computes the ℓ2-regularizer. It calculates 0.5 times λ times the dot product of w with itself (transposed). This is the squared ℓ2-norm of w, scaled by λ.

g = lbda * w: This line computes the gradient of the ℓ2-regularizer. It's simply λ times w, as the derivative of 0.5 * λ * ||w||² with respect to w is just λw.

This function computes both the ℓ2-regularizer r and its gradient g and returns them.

### Exercise 4

Fill in the code for the function predict(w,x) which predicts the class label $y$ for a data point $\textbf{x}$ or a matrix $X$ of data points (row-wise) for a previously trained linear model $\textbf{w}$. If there is only a data point given, the function is supposed to return a scalar value. If a matrix is given a vector of predictions is supposed to be returned.

In [5]:
import numpy as np

def predict(w, X):
    preds = np.sign(np.dot(X, w))  # Predictions: 1 if dot product >= 0, -1 otherwise
    return preds


Explanation:

np.dot(X, w): This computes the dot product of the data points (X) with the weight vector (w). For a single data point, this results in a scalar; for a matrix of data points, this results in a vector of dot products.

np.sign(...): This function returns the sign of each element in the input array. So, if the dot product is positive or zero, it returns 1; otherwise, it returns -1.

This function returns the predictions for the given data points based on the previously trained linear model
�
w.